In [ ]:
# Modules
import gurobipy as grb
import pandas as pd
import numpy as np
import math
import random

# Standard output divider
div = "-----"

# Print warning message


# Timeslots
alltimeslots = {
   # 0: "8:00am",
    1: "9:00am",
    2: "10:00am",
    3: "11:00am",
    4: "12:00pm",
    5: "1:00pm",
    6: "2:00pm",
    7: "4:00pm",
    8: "5:00pm",
}

alltimeslots_inv = {
    "9am-10am":1,
    "10am-11am":2,
    "11am-12pm":3,
    "12pm-1pm":4,
    "1pm-2pm":5,
    "2pm-3pm":6,
    "4pm-5pm":7,
    "5pm-6pm":8,
}


# Dictionary of speaker vetos and corresponding values

prefs = {
    0:"No Preference",
    1:"1st Preference",
    2:"2nd Preference",
    3:"3rd Preference",
    4:"4th Preference",
    5:"5th Preference",
}

prefsInv = {
    "No Preference":0,
    "1st preference":1,
    "2nd Preference":2,
    "3rd Preference":3,
    "4th Preference":4,
    "5th Preference":5,
}

sides = ["aff","neg"]

# Person class
# - No preferences or vetoes for Easters trials

class person:
    name = ""
    availabilities = {}
    preferences = {}
    
    def __init__(self,name,availabilities,prefs_aff,prefs_neg,isAllocated):
        self.name = name
        self.availabilities = availabilities
        self.preferences["aff"] = prefs_aff
        self.preferences["neg"] = prefs_neg
        self.isAllocated = False
        #print(self.preferences)
        
        
def importData():
    print("Please note the final allocation may take up to 10 minutes to compute.")
    global speakers
    speakers = []
    global times
    times = {}
    # CSV reading and parsing
    global csv
    csv = pd.read_csv("unsw_novs.csv")

    for index,row in csv.iterrows():
        name = row[1]
        av = row[18].split(", ")
        for i in range(len(av)):
            av[i] = alltimeslots_inv[av[i]]
        prefs_aff = []
        prefs_neg = []
    
        prefs_aff.append(prefsInv[row[12]])
        prefs_neg.append(prefsInv[row[13]])
        prefs_aff.append(prefsInv[row[14]])
        prefs_neg.append(prefsInv[row[15]])
        prefs_aff.append(prefsInv[row[16]])
        prefs_neg.append(prefsInv[row[17]])
        
        speakers.append(person(name,av,prefs_aff,prefs_neg,False)) # MARK: Details

    
    '''If the allocator produces an infeasible result there's a good chance it needs more timeslots.
    Manually override times with a higher number of slots if required'''
    
    # Form the feasible team set
    global ftset
    global ftsample
    ftset = {}
    ftsample = {}
    totalteams = len(speakers)**3
    index = 1
    print("Forming Feasible Team Set...")
    
    
    for speaker1 in speakers:
        for speaker2 in speakers:
            for speaker3 in speakers:
                if speaker1 != speaker2 and speaker1 != speaker3 and speaker2 != speaker3:
                    #print("Constructing team {} of {}...".format(index,totalteams))
                    ftset[index] = [speaker1,speaker2,speaker3]
                    index += 1
    '''
    for speaker1 in speakers:
        for speaker2 in speakers:
            if speaker1 != speaker2:
                #print("Constructing team {} of {}...".format(index,totalteams))
                ftset[index] = [speaker1,speaker2]
                index += 1
    '''              
                    
    randomkeys = random.sample(ftset.keys(), len(ftset)/500) # Vary so that a smaller subset is used for the fts
    for k in randomkeys:
        ftsample[k] = ftset[k]
        
    #times = alltimeslots
    times = {
        0: "8:00am",
        2: "10:00am",
        3: "11:00am",
        4: "12:00pm",
        5: "1:00pm",
        7: "4:00pm",
        8: "5:00pm",
    }
    
def calculateCosts():
    # Costs
    global costs
    costs = {}
    totalcosts = len(ftsample)*len(times)*2
    print("Calculating costs...")
    index2 = 1
    for key,team in ftsample.iteritems():
        for slot in times.keys():
            for side in sides:
                costs[key,slot,side] = 0
                index2 += 1
                #print("Calculating cost {} of {}...".format(index2,totalcosts))
    
    
                # Position preferences

                # Timing vetoes
                cost = 0
                for speaker in team:
                    pos = team.index(speaker)
                    cost += speaker.preferences[side][pos]**5
                    if slot not in speaker.availabilities:
                        cost += 1000#cost = grb.GRB.INFINITY
                costs[key,slot,side] = cost
            
            
def allocate():
    # Optimisation
    a = grb.Model("2019 Easters Trial Allocator")
    x = {}
    for key,value in costs.iteritems():
        x[key] = a.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(key))
        
    # Exactly 1 team of each side to each timeslot
    for slot in times.keys():
        #for side in sides:
        a.addConstr(sum(x[team,slot,side] for team in ftsample.keys() for side in sides) == 2)
        
    # Each team to no more than 1 timeslot
    for key,team in ftsample.iteritems():
        a.addConstr(sum(x[key,slot,side] for slot in times.keys() for side in sides) <= 1)

    for s in speakers:
        affectedTeams = []
        for key,team in ftsample.iteritems():
            if s in team:
                affectedTeams.append(key)
                
        # Each speaker to no more than 1 debate in 1 timeslot
        '''Can be 0 debates (ie. unallocated) and then appended
        as alternate 3rds'''
        a.addConstr(sum(x[team,slot,side] for team in affectedTeams for slot in times.keys() for side in sides) <= 1)
    
    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
    allocated = []
    unallocated = []
    unallocatednames = []
    print(div)
    print("UNSW EASTERS TRIALS 2019 - DEBATE ALLOCATION")
    print("Speakers in each team are listed in speaking order.")
    #print("Sides for each debate should be determined randomly.")
    print(div)
    for key,value in costs.iteritems():
        if x[key].x != 0:
            spk = []
            for s in ftsample[key[0]]:
                spk.append(s.name)
            print("Team {} {} on side {} at {} (Cost {})".format(key[0],spk,key[2].upper(),times[key[1]],value))
            for speaker in ftsample[key[0]]:
                speaker.isAllocated = True
                allocated.append(speaker.name)
    for speaker in speakers:
        if speaker.name not in allocated:#if speaker.isAllocated == False:
            unallocated.append(speaker)
            unallocatednames.append(speaker.name)
    print("Total Cost: {}".format(a.objVal))
    print(div)
    
    j = ", "
    print("Unallocated Speakers: {}".format(j.join(unallocatednames)))
    print(div)
    #report(unallocated)

def report(speakers):
    for speaker in speakers:
        if speaker not in speakers:
            print("Speaker {} does not exist".format(speaker.name))
        else:
            vetoString= "{} {}.".format(speaker.name, vetodictinv[speaker.speakerveto])

            print("Report for {}".format(speaker.name))
            print(vetoString)
            print("{}'s availability is as follows:".format(speaker.name))
            for slot,availability in speaker.availabilities.iteritems():
                try:
                    # TODO
                    pass
                    #print("{}: {}".format(times[slot],availability))
                except:
                    pass
        print(div)

In [ ]:
importData()

In [ ]:
calculateCosts()
for key,cost in costs.iteritems():
    print(key,cost)
#allocate()

In [ ]:
# Symeon and Sam Braham want to trial by CV

allocate()

In [ ]:
print(costs)